In [1]:
import sys,os,os.path
#sys.path.append("../../")   # cite IC from parent directory
sys.path.append("/gluster/data/next/software/IC_satkill/")
#sys.path.append("/gluster/data/next/software/IC_sophronia/")
#sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
os.environ['ICTDIR']='/gluster/data/next/software/IC_satkill/'


#%load_ext autoreload
#%autoreload 2
#%matplotlib notebook

import time

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['mathtext.fontset'] = 'stix'
rcParams['font.family'] = 'STIXGeneral'
rcParams['figure.figsize'] = [10, 8]
rcParams['font.size'] = 22

import pandas as pd
import numpy  as np
import tables as tb

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.colors as clrs
from matplotlib.lines import Line2D


import IC.invisible_cities.core.core_functions                   as     coref
import IC.invisible_cities.io.dst_io                           as     dstio

from IC.invisible_cities.cities                 import beersheba as beerfun

from IC.invisible_cities.evm.event_model                          import HitCollection

from IC.invisible_cities.database.load_db       import DataSiPM

from IC.invisible_cities.evm.event_model        import Cluster, Hit
from IC.invisible_cities.types.ic_types         import xy
from IC.invisible_cities.reco.paolina_functions import voxelize_hits, drop_end_point_voxels, make_track_graphs, get_track_energy

from IC.invisible_cities.evm.event_model        import HitEnergy
from IC.invisible_cities.cities.beersheba          import DeconvolutionMode
from IC.invisible_cities.cities.beersheba          import CutType

from IC.invisible_cities.reco import hits_functions as hif

from IC.invisible_cities.reco.deconv_functions import deconvolve
from IC.invisible_cities.reco.deconv_functions import deconvolution_input
from IC.invisible_cities.reco.deconv_functions import InterpolationMethod

import IC.invisible_cities.io.mcinfo_io as mcio

from collections import defaultdict

import matplotlib.cm as cm
from matplotlib.colors import Normalize


In [9]:
file_no = 30
file = f'/gluster/data/next/files/TOPOLOGY_John/HYPPOS_DATA_QTHR/Q_THR4/PORT_1a/sophronia/sophronia_' + str(file_no) + '_208Tl.h5'

In [10]:
MC_df = pd.read_hdf(file, 'MC/particles')

In [15]:
#display(MC_df)
evt_id = MC_df[MC_df.particle_name == 'e+'].event_id.to_numpy()

In [16]:
print(evt_id)

[290013 290031 290032 290038 290044 290048 290051 290055 290058 290073
 290078 290081 290083 290099 290106 290108 290110 290112 290114 290125
 290140 290155 290159 290166 290173 290178 290215 290216 290218 290230
 290253 290261 290273 290278 290292 290304 290310 290315 290337 290342
 290353]


In [18]:
pd.set_option('display.max_rows', 500)
display(MC_df[MC_df.event_id == evt_id[0]])

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
1044,290013,1,Tl208,True,0,396.107330,393.276245,534.039307,0.000000,396.107330,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,none,RadioactiveDecay
1045,290013,4,e-,False,1,396.107330,393.276245,534.039307,0.000183,404.240479,...,0.779324,1.095760,-0.359691,-0.000000,0.000000,0.000000,0.971746,1.545165e+01,RadioactiveDecay,eIoni
1046,290013,3,anti_nu_e,False,1,396.107330,393.276245,534.039307,0.000183,-8222.476562,...,-0.316265,0.444264,0.085812,-0.316265,0.444264,0.085812,0.552049,1.504398e+04,RadioactiveDecay,Transportation
1047,290013,2,Pb208[3475.078],False,1,396.107330,393.276245,534.039307,0.000183,396.107330,...,-0.463059,-1.540024,0.273879,-0.000000,-0.000000,0.000000,0.000007,0.000000e+00,RadioactiveDecay,RadioactiveDecay
1048,290013,6,gamma,False,2,396.107330,393.276245,534.039307,0.000183,460.731201,...,-0.527390,-0.391711,0.555856,0.000000,0.000000,-0.000000,0.860554,2.767462e+02,RadioactiveDecay,phot
1049,290013,21,e-,False,6,460.731201,331.111450,540.388916,0.923309,460.731201,...,-0.005523,-0.005359,0.000156,-0.000000,-0.000000,0.000000,0.000058,1.447787e-07,phot,msc
1050,290013,20,e-,False,6,460.731201,331.111450,540.388916,0.923309,460.731201,...,0.006145,-0.002717,-0.004100,-0.000000,0.000000,0.000000,0.000061,1.473075e-07,phot,msc
1051,290013,19,e-,False,6,460.731201,331.111450,540.388916,0.923309,460.731201,...,0.008027,0.014151,-0.023069,0.000000,0.000000,-0.000000,0.000779,1.385419e-05,phot,eIoni
1052,290013,18,gamma,False,6,460.731201,331.111450,540.388916,0.923309,460.740753,...,0.006884,0.002067,0.003525,0.000000,-0.000000,0.000000,0.008006,1.171419e-02,phot,phot
1053,290013,24,e-,False,18,460.740753,331.106476,540.393433,0.923348,460.740753,...,-0.022438,0.020111,-0.005542,-0.000000,0.000000,-0.000000,0.000918,1.704574e-05,phot,eIoni


### Need a bit of code to check and make sure that the parent particle of these positrons is Xe136

In [61]:
# select all positrons

# create tuple of event_id - mother_id to select from

positron_event_mothers = ((MC_df[MC_df.particle_name == 'e+']).event_id.to_numpy(), (MC_df[MC_df.particle_name == 'e+']).mother_id.to_numpy())
display(positron_event_mothers[0][10])
display(positron_event_mothers[1][10])
display(MC_df[MC_df.event_id == 290078])
display(MC_df[MC_df.particle_name == 'Xe131'])

290078

44

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
6379,290078,1,Tl208,True,0,397.517181,393.651123,533.796814,0.000000,397.517181,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,none,RadioactiveDecay
6380,290078,4,e-,False,1,397.517181,393.651123,533.796814,0.000370,396.172668,...,0.226810,-0.541064,-0.082542,0.000000,0.000000,-0.000000,2.713859e-01,3.407538e+00,RadioactiveDecay,eIoni
6381,290078,3,anti_nu_e,False,1,397.517181,393.651123,533.796814,0.000370,-906.634766,...,-0.108366,-0.753754,0.994305,-0.108366,-0.753754,0.994305,1.252410e+00,1.507243e+04,RadioactiveDecay,Transportation
6382,290078,2,Pb208[3475.078],False,1,397.517181,393.651123,533.796814,0.000370,397.517181,...,-0.118445,1.294818,-0.911763,-0.000000,0.000000,-0.000000,6.510090e-06,0.000000e+00,RadioactiveDecay,RadioactiveDecay
6383,290078,6,gamma,False,2,397.517181,393.651123,533.796814,0.000370,414.921906,...,0.272301,-0.010746,-0.051663,0.000000,-0.000000,-0.000000,2.773668e-01,1.772853e+01,RadioactiveDecay,phot
6384,290078,12,e-,False,6,414.921906,392.964264,530.494629,0.059506,414.921906,...,0.002271,-0.007032,0.002156,0.000000,-0.000000,-0.000000,5.797000e-05,1.447787e-07,phot,msc
6385,290078,11,e-,False,6,414.921906,392.964264,530.494629,0.059506,414.921906,...,-0.004965,-0.005903,0.000208,-0.000000,-0.000000,0.000000,5.826000e-05,1.450558e-07,phot,msc
6386,290078,10,e-,False,6,414.921906,392.964264,530.494629,0.059506,414.921906,...,-0.002859,0.000033,-0.005115,-0.000000,0.000000,-0.000000,3.360000e-05,1.221262e-07,phot,msc
6387,290078,9,e-,False,6,414.921906,392.964264,530.494629,0.059506,414.921906,...,0.018118,-0.018195,-0.009791,0.000000,-0.000000,-0.000000,7.383700e-04,1.259782e-05,phot,eIoni
6388,290078,8,gamma,False,6,414.921906,392.964264,530.494629,0.059506,414.936340,...,0.001473,0.002954,0.007294,0.000000,0.000000,0.000000,8.005710e-03,7.840825e-02,phot,phot


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
3120,290038,28,Xe131,False,25,-329.410553,152.934982,779.154663,2.691185,-329.410675,...,-0.342947,-0.645131,-0.134031,-0.0,-0.0,-0.0,0.000002,0.000309,conv,NoProcess
14488,290178,66,Xe131,False,63,186.531647,109.520004,773.333679,1.423868,186.531677,...,0.052582,-0.006367,0.990931,0.0,-0.0,0.0,0.000004,0.000412,conv,NoProcess
17632,290218,43,Xe131,False,40,-86.615578,82.166649,588.105042,1.924311,-86.615837,...,-0.629226,0.011447,-0.349185,-0.0,0.0,-0.0,0.000002,0.000299,conv,NoProcess
22073,290273,32,Xe131,False,29,108.491875,381.571106,885.146973,1.517249,108.491722,...,-0.366685,-0.736688,0.442954,-0.0,-0.0,0.0,0.000004,0.000388,conv,NoProcess


In [62]:
MC_df.final_proc.unique()

array(['RadioactiveDecay', 'eIoni', 'phot', 'msc', 'Transportation',
       'NoProcess', 'conv', 'annihil', 'UserSpecialCut'], dtype=object)

In [63]:
MC_df.creator_proc.unique()

array(['none', 'RadioactiveDecay', 'eBrem', 'phot', 'compt', 'eIoni',
       'conv', 'annihil'], dtype=object)

In [ ]:
MC_df[MC_df.creator_proc == 'conv']